In [ ]:
#mount google drive
from google.colab import drive

# Mount the drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install dependents
%pip install openai
%pip install chromadb
%pip install langchain
%pip install --upgrade --quiet pymupdf
%pip install -U langchain-community
%pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.4 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyMuPDFLoader

In [ ]:
# set the path for all the files

In [ ]:
# change pdf into XML if necessary

In [ ]:
# load the file
PDF_NAME='/content/drive/My Drive/ChenPark2020.pdf'
def load_pdf():
  return PyMuPDFLoader(PDF_NAME).load()

In [ ]:
docs = load_pdf()

In [ ]:
print (f'You have {len(docs)} document(s) in your data')
print (f'There are {len(docs[0].page_content)} characters in the first page of your document')

You have 19 document(s) in your data
There are 3852 characters in the first page of your document


In [ ]:
print(docs[0].page_content)

Computers & Education 150 (2020) 103836
Available online 5 February 2020
0360-1315/© 2020 Elsevier Ltd. All rights reserved.
Contents lists available at ScienceDirect
Computers & Education
journal homepage: www.elsevier.com/locate/compedu
Teaching and learning with children: Impact of reciprocal peer
learning with a social robot on children’s learning and emotive
engagement
Huili Chen ∗, Hae Won Park, Cynthia Breazeal
Personal Robots Group, MIT Media Lab, Cambridge, MA, USA
A R T I C L E
I N F O
Keywords:
Intelligent tutoring systems
Interactive learning environments
Evaluation of CAL systems
Cooperative/collaborative learning
A B S T R A C T
Pedagogical agents are typically designed to take on a single role: either as a tutor who
guides and instructs the student, or as a tutee that learns from the student to reinforce what
he/she knows. While both agent-role paradigms have been shown to promote student learning,
we hypothesize that there will be heightened benefit with respect to stud

In [ ]:
total = 0
for doc in docs:
  total += len(doc.page_content)
print (f'There are {total} characters in your document')

There are 97663 characters in your document


In [ ]:
# splite
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

In [ ]:
# initial a embedding
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import os

OPENAI_API_KEY = 

In [ ]:
# initial a vector database
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small")

In [ ]:
# persist the vector database to disk
persist_directory = '/content/drive/My Drive/meta'
collection_name = 'meta_index'

In [ ]:
# LangChain OpenAI bill monitor
from langchain.callbacks import get_openai_callback

In [ ]:
# call openAI for vectorbase embedding
with get_openai_callback() as cb:
    vectorstore = Chroma.from_documents(split_docs, embeddings, collection_name=collection_name, persist_directory=persist_directory)
    vectorstore.persist()
    print(cb)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [ ]:
# retrieval information from the vector database
# Load the vectorstore from disk
vectorstore = Chroma(collection_name=collection_name, persist_directory=persist_directory, embedding_function=embeddings)

query = "What is intelligent tutoring systems?"
docs = vectorstore.similarity_search(query, 3)

In [ ]:
for doc in docs:
    print(doc.metadata)

{'author': 'Huili Chen', 'creationDate': '4th March 2020', 'creator': 'Elsevier', 'file_path': '/content/drive/My Drive/ChenPark2020.pdf', 'format': 'PDF 1.7', 'keywords': 'Intelligent tutoring systems,Interactive learning environments,Evaluation of CAL systems,Cooperative/collaborative learning', 'modDate': "D:20240211153223-05'00'", 'page': 17, 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'source': '/content/drive/My Drive/ChenPark2020.pdf', 'subject': 'Computers & Education, 150 (2020) 103836. doi:10.1016/j.compedu.2020.103836', 'title': 'Teaching and learning with children: Impact of reciprocal peer learning with a social robot on children’s learning and emotive engagement', 'total_pages': 19, 'trapped': ''}
{'author': 'Huili Chen', 'creationDate': '4th March 2020', 'creator': 'Elsevier', 'file_path': '/content/drive/My Drive/ChenPark2020.pdf', 'format': 'PDF 1.7', 'keywords': 'Intelligent tutoring systems,Interactive learning environments,Evaluation of CAL systems,Cooperative/

In [ ]:
# chat bot
from langchain_community.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY, model_name="gpt-4o-mini-2024-07-18")

chain = load_qa_chain(llm, chain_type="stuff")

with get_openai_callback() as cb:
    print(chain.run(input_documents=docs, question=query))
    print(cb)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Intelligent tutoring systems (ITS) are computer-based educational systems designed to provide personalized instruction and feedback to students in a one-on-one interaction. They aim to support students across various ages and academic domains by assessing mastery, modeling cognitive states, adapting content to individual needs, and considering student affect and motivation. ITS are inspired by the 2-sigma effect, which highlights the effectiveness of one-on-one tutoring compared to traditional classroom instruction.
Tokens Used: 832
	Prompt Tokens: 746
	Completion Tokens: 86
Successful Requests: 1
Total Cost (USD): $0.0001635


In [ ]:
# build prompt

In [ ]:
prompt1 = "Does the article discuss an original structured investigation in the form of experiments, trials, or treatments?"
prompt2 = "assessed treatment outcomes after participants interact with [your application/tool/medicine etc.]"
prompt3 = "compared outcomes of [your application/tool/medicine etc.] from the experiment group with outcomes from a non-treatment mode from the control/comparison group"
prompt4 = "reported sufficient data to calculate effect size"
prompt5 = "reported measurable outcomes that can be further analyzed"

In [ ]:
prompt = [prompt1, prompt2, prompt3, prompt4, prompt5]


In [ ]:
query = prompt1
docs = vectorstore.similarity_search(query, 3)
with get_openai_callback() as cb:
    print(chain.run(input_documents=docs, question=query))
    print(cb)

Yes, the article discusses a structured investigation in the form of a between-subjects experiment designed to explore how different child-agent interaction paradigms influence children's learning and affective experience.
Tokens Used: 780
	Prompt Tokens: 744
	Completion Tokens: 36
Successful Requests: 1
Total Cost (USD): $0.00013319999999999999


In [ ]:
for doc in docs:
    print(doc.page_content)

conversational exposure is associated with language-related brain function. Psychological Science, 29(5), 700–710, PMID: 29442613. http://dx.doi.org/10.
1177/0956797617742725.
Ros, R., Oleari, E., Pozzi, C., Sacchitelli, F., Baranzini, D., Bagherzadhalimi, A., et al. (2016). A motivational approach to support healthy habits in long-term
child–robot interaction. International Journal of Social Robotics.
Roscoe, R. D., & Chi, M. T. H. (2007). Understanding tutor learning: Knowledge-Building and knowledge-telling in peer tutors explanations and questions. Review
of Educational Research, 77(4), 534–574.
Rubin, K. H., Bukowski, W. M., & Parker, J. G. (2007). Peer interactions, relationships, and groups. In Handbook of child psychology.
Ryokai, K., Boulanger, C., & Cassell, J. (2003). Virtual peers as partners in storytelling and literacy learning. Journal of Computer Assisted Learning, 19, 195–208.
of social learning and the psychology of social engagement to benefit not only skill learning